In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re
import time

# Load the data

In [2]:
directory='./data/'

In [3]:
appdesc_names=['application_version_id','package_name','name','version','developer','app_type','category','m2_category',\
               'permissions','permission_weight','notification_score','analytic_providers','apk_size','ignore_category',\
               'icon_link', 'not_on_play','create_device_id','create_date','update_date']

In [4]:
appdesc_df=pd.read_csv(directory+'application_versions.csv', error_bad_lines=False, names=appdesc_names)

In [5]:
###########################
# Lowercase
appdesc_df['name']=appdesc_df['name'].map(lambda x: str(x).lower())

###########################
# Remove special characters
appdesc_df['name']=appdesc_df['name'].map(lambda x: re.sub(r'([^\s\w]|_)+', '', x))

# Get time spent per category for each user

### Concatenate all months

filenames = ['./data/app_usage_events_all_september.csv','./data/app_usage_events_all_october.csv','./data/app_usage_events_all_november.csv']
with open('./data/app_usage_events_all', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            if fname != './data/app_usage_events_all_september.csv':
                infile.readline()
            for line in infile:
                outfile.write(line)

### Get user data

In [6]:
def get_category_integer(package_name):
    try:
        category_integer=int(appdesc_df[appdesc_df['package_name']==package_name]['category'].values[0])
    except:
        return 0
    if category_integer<0:
        category_integer=0
    return category_integer

In [7]:
total_lines=166415646
#total_lines=10000
lines_per_read=1000000

header_list=['device_id','application_version_id','type','start_date','run_time','end_date','continuation','year',\
             'month','day','package_name']

device_id_index=0
package_name_index=10
run_time_index=4

In [8]:
null_line={'device_id': '0'}
total_categories=49
for i in range(total_categories):
    null_line[str(i)]=i
    
user_df=pd.DataFrame(data=null_line, index=[0])
cols=(['device_id']+[str(i) for i in range(total_categories)])
user_df=user_df[cols]

In [10]:
pickle_filename='./data/pkl/user_data.pkl'

In [11]:
i=1
while i < total_lines:
    t1=time.time()
    skiprows=i
    if skiprows+lines_per_read > total_lines:
        nrows=total_lines-skiprows
    else:
        nrows=lines_per_read
        
    temp_df=pd.read_csv(directory+'app_usage_events_all.csv', skiprows=skiprows, nrows=nrows, names=header_list)
    temp_df=temp_df[temp_df['type']!=4][['device_id', 'package_name', 'run_time']]
    temp_df['category']=temp_df['package_name'].map(lambda x: get_category_integer(x))
    temp_df=temp_df[temp_df['category']!=0]
    
    for device_id in temp_df['device_id'].unique():
        temp_temp_df=temp_df[temp_df['device_id']==device_id]
        totals_dict={}
        totals_dict['device_id']=device_id
        totals_list=[0 for i in range(total_categories+1)]
        totals_list[0]=device_id
        
        for i in range(total_categories):
            totals_dict[str(i)]=0

        for category in np.sort(temp_temp_df['category'].unique()):
            category_total=temp_temp_df[temp_temp_df['category']==category]['run_time'].sum()
            totals_dict[category]=category_total
            totals_list[category]=category_total
            
        user_df.loc[len(user_df)]=totals_list
        
    i=skiprows+nrows+1
    t2=time.time()
    print 'i=',i,'dt=',t2-t1
    user_df.to_pickle(pickle_filename)

i= 1000002 dt= 424.782610893
i= 2000003 dt= 403.753925085
i= 3000004 dt= 414.778650045
i= 4000005 dt= 319.298412085
i= 5000006 dt= 356.618561029
i= 6000007 dt= 398.31009984
i= 7000008 dt= 355.729816914
i= 8000009 dt= 475.829794884
i= 9000010 dt= 383.039916992
i= 10000011 dt= 379.922968864
i= 11000012 dt= 381.120043993
i= 12000013 dt= 347.118180037
i= 13000014 dt= 406.783582926
i= 14000015 dt= 387.595858812
i= 15000016 dt= 427.790014982
i= 16000017 dt= 440.221220016
i= 17000018 dt= 422.063371897
i= 18000019 dt= 334.282606125
i= 19000020 dt= 339.053781033
i= 20000021 dt= 390.280629873
i= 21000022 dt= 320.772987127
i= 22000023 dt= 460.355503082
i= 23000024 dt= 320.106631994
i= 24000025 dt= 379.306640863
i= 25000026 dt= 448.717424154
i= 26000027 dt= 494.071552992
i= 27000028 dt= 508.266517162
i= 28000029 dt= 469.319725037
i= 29000030 dt= 382.673518181
i= 30000031 dt= 638.043957949
i= 31000032 dt= 543.993613958
i= 32000033 dt= 488.832910061
i= 33000034 dt= 489.615074873
i= 34000035 dt= 450.

KeyboardInterrupt: 